In [1]:
import torch
from transformers import BertTokenizer, AdamW, BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer, AutoModel
import tqdm
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import torchtext
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import jieba
import os

I0415 13:49:10.925303 140541114541824 file_utils.py:41] PyTorch version 1.2.0 available.


# dataset

In [2]:
class CLSDataset(Dataset):
    def __init__(self, df, is_train = True):
    # 加载语料
        # 将数据集全部加载到内存
        self.df = df.copy()
        self.df = self.df.reset_index()

        self.is_train = is_train
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.is_train:
            query_id   = self.df.loc[idx, '微博id']
            input_ids = self.df.loc[idx, 'input_ids']
            token_type_ids = self.df.loc[idx, 'token_type_ids']
            attention_mask = self.df.loc[idx, 'attention_mask']
            label = self.df.loc[idx, '情感倾向']

            sample = {"query_id":query_id,
                      "input_ids":input_ids, 
                      "token_type_ids":token_type_ids , 
                      "attention_mask":attention_mask,
                      "label": label}
            return sample           
        else:
            query_id   = self.df.loc[idx, '微博id']
            input_ids = self.df.loc[idx, 'input_ids']
            token_type_ids = self.df.loc[idx, 'token_type_ids']
            attention_mask = self.df.loc[idx, 'attention_mask']

            sample = {"query_id":query_id,
                      "input_ids":input_ids, 
                      "token_type_ids":token_type_ids , 
                      "attention_mask":attention_mask}
            return sample   

In [3]:
def collate_fn(batch):
    # 因为token_list是一个变长的数据，所以需要用一个list来装这个batch的token_list
    # 
    query_id = [item['query_id'] for item in batch]
    input_ids = [eval(item['input_ids']) for item in batch]
    token_type_ids = [eval(item['token_type_ids']) for item in batch]
    attention_mask = [eval(item['attention_mask']) for item in batch]
    label = [item['label'] for item in batch]
    return {
        'query_id':query_id,
        'input_ids': input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask' : attention_mask,
        'label' : label
            }

def collate_dev_test_fn(batch):
    # 因为token_list是一个变长的数据，所以需要用一个list来装这个batch的token_list
    # 
    query_id = [item['query_id'] for item in batch]
    input_ids = [eval(item['input_ids']) for item in batch]
    token_type_ids = [eval(item['token_type_ids']) for item in batch]
    attention_mask = [eval(item['attention_mask']) for item in batch]
    return {
        'query_id':query_id,
        'input_ids': input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask' : attention_mask,
            }

# model

In [45]:
class CLS_model(nn.Module):
    def __init__(self, embedding_dim, target_size):
        super(CLS_model, self).__init__()
#         self.model = AutoModel.from_pretrained('/home/zhoujx/Pretrained_models/chinese_xlnet_base_pytorch')
        self.bert_config = BertConfig.from_pretrained('/home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/bert_config.json', output_hidden_states=True)
        self.tokenizer  = BertTokenizer.from_pretrained('/home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/vocab.txt', config=self.bert_config)
        self.model = BertModel.from_pretrained('/home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch', config=self.bert_config)
        self.fc1 = nn.Linear(embedding_dim*2, embedding_dim)
        self.activation1 = nn.Tanh()
        self.fc2 = nn.Linear(embedding_dim, target_size)
        
    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None):
        last_hidden_state, pooled_output, _ = self.model(input_ids, token_type_ids=token_type_ids, attention_mask = attention_mask)
#         last_hidden_state = self.model(input_ids, attention_mask = attention_mask)
        seq_avg = torch.mean(last_hidden_state, dim=1)
        seq_max = torch.max(last_hidden_state, dim=1)[0]
        concat_out = torch.cat((seq_avg, seq_max), dim=1)
        fc1_out = self.activation1(self.fc1(concat_out))
        fc2_out = self.fc2(fc1_out)
        
        return  fc2_out

    def predict(self, sentence):
        input_ids = self.tokenizer.encode(sentence)
        input_ids = torch.LongTensor(input_ids)
        input_ids = input_ids.unsqueeze(0).to(device)
        output = self.forward(input_ids)
        prediction = torch.max(output, dim=1)[1].cpu().numpy()
        if prediction == 1:
            return '中立'
        elif prediction == 0:
            return '悲观'
        else :
            return '乐观'

# uxiliary Function

## count_parameters

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f'The model has {count_parameters(model):,} trainable parameters')

## evaluate

In [6]:
def evaluate(all_predictions, all_labels):
    print('\tprecison_score：', precision_score(all_predictions, all_labels, average=None, labels=[0,1,2]))
    print('\trecall_score：', recall_score(all_predictions, all_labels, average=None, labels=[0,1,2]))
    print('\tf1_score：', f1_score(all_predictions, all_labels, average=None, labels=[0,1,2]))
    print('\tf1_score：', f1_score(all_predictions, all_labels, average='macro', labels=[0,1,2]))
    score_f1 = f1_score(all_predictions, all_labels, average='macro', labels=[0,1,2])
    return score_f1

## PGD

In [7]:
class PGD():
    def __init__(self, model):
        self.model = model
        self.emb_backup = {}
        self.grad_backup = {}

    def attack(self, epsilon=1., alpha=0.3, emb_name='word_embeddings', is_first_attack=False):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if is_first_attack:
                    self.emb_backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = alpha * param.grad / norm
                    param.data.add_(r_at)
                    param.data = self.project(name, param.data, epsilon)

    def restore(self, emb_name='word_embeddings'):
        # emb_name这个参数要换成你模型中embedding的参数名
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.emb_backup
                param.data = self.emb_backup[name]
        self.emb_backup = {}

    def project(self, param_name, param_data, epsilon):
        r = param_data - self.emb_backup[param_name]
        if torch.norm(r) > epsilon:
            r = epsilon * r / torch.norm(r)
        return self.emb_backup[param_name] + r

    def backup_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                try:
                    self.grad_backup[name] = param.grad.clone()
                except:
                    pass

    def restore_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                try:
                    param.grad = self.grad_backup[name]
                except:
                    pass

# train

In [ ]:
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 训练过程
    # 五折训练
    # 训练集
    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=4)
    df = pd.read_csv(r'./corpus/df_train.csv')
    print(df.shape)
        
    for fold, (train_index, valid_index) in enumerate(skf.split(df, df.情感倾向)):
        # 训练集
        df_train = df.iloc[train_index]
        train_dataset = CLSDataset(df_train)
        train_dataloader = DataLoader(train_dataset,
                                      shuffle=True,
                                      batch_size=4,
                                      collate_fn = collate_fn)
        # 测试集
        df_val = df.iloc[valid_index]
        dev_dataset = CLSDataset(df_val)
        dev_dataloader = DataLoader(dev_dataset,
                                     shuffle=False,
                                     batch_size=12,
                                     collate_fn = collate_fn)
        
        model = CLS_model(embedding_dim = 1024, target_size = 3).to(device)
        # 对抗学习层
        pgd = PGD(model)
        K = 3
        # 
        print(count_parameters(model))
        optimizer = AdamW(params=model.parameters(), lr=1e-6)
        criterion = nn.CrossEntropyLoss().to(device)

        best_f1 = 0
        PATH = './output/bert_large_pag_{}.pkl'.format(fold)
    
        for epoch in range(3):
            all_predictions = []
            all_labels = []
            model.train()
            pbar = tqdm.tqdm(enumerate(train_dataloader), 
                             miniters = 50,
                             desc='fold：{}__epoch：{}__train'.format(fold, epoch),
                             total=len(train_dataloader), 
                             ncols=50)
            with  pbar as t:
                for idx, batch_samples in t:    
                    query_id = batch_samples['query_id']             
                    input_ids = batch_samples['input_ids']
                    token_type_ids = batch_samples['token_type_ids']
                    attention_mask = batch_samples['attention_mask']
                    label = batch_samples['label']
                    
                    #
                    input_ids = torch.LongTensor(input_ids).to(device)
                    token_type_ids = torch.LongTensor(token_type_ids).to(device)
                    attention_mask = torch.LongTensor(attention_mask).to(device)
                    label = torch.LongTensor(label).to(device)
                    
                    # 前向传播
                    predictions = model(input_ids, 
                                       token_type_ids = token_type_ids,
                                       attention_mask = attention_mask) 
                    loss = criterion(predictions, label)
                    loss.backward()
                    pgd.backup_grad()
                    # 对抗训练
                    for t in range(K):
                        pgd.attack(is_first_attack=(t == 0))  # 在embedding上添加对抗扰动, first attack时备份param.data
                        if t != K - 1:
                            model.zero_grad()
                        else:
                            pgd.restore_grad()
                        y_pred = model(input_ids, 
                                       token_type_ids = token_type_ids,
                                       attention_mask = attention_mask) 
                        loss_adv = criterion(y_pred, label)
                        loss_adv.backward()  # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
                    pgd.restore()  # 恢复embedding参数
                    # 梯度下降，更新参数
                    optimizer.step()
                    model.zero_grad()

                    # evaluate
                    _, pre_labels = torch.max(predictions.squeeze(1), dim=1)
                    pre_labels = pre_labels.detach().cpu().numpy().reshape(-1).tolist()
                    all_predictions.extend(pre_labels)
                    all_labels.extend(label.cpu().numpy().tolist())

                    if idx % 50 == 0:
                        print(Counter(all_predictions))
                        print(Counter(all_labels))
                        evaluate(all_predictions, all_labels)

            #每一个epoch打印验证集结果
            model.eval()
            all_dev_predictions, all_dev_labels = [], []
            for batch_samples in dev_dataloader:    
                query_id = batch_samples['query_id']             
                input_ids = batch_samples['input_ids']
                token_type_ids = batch_samples['token_type_ids']
                attention_mask = batch_samples['attention_mask']
                label = batch_samples['label']

                #
                input_ids = torch.LongTensor(input_ids).to(device)
                token_type_ids = torch.LongTensor(token_type_ids).to(device)
                attention_mask = torch.LongTensor(attention_mask).to(device) 

                with torch.no_grad():
                    predictions = model(input_ids, 
                                        attention_mask = attention_mask) 
                    pre_labels = torch.max(predictions.squeeze(1), dim=1)[1]
                    pre_labels = pre_labels.detach().cpu().numpy().reshape(-1).tolist()
                    all_dev_predictions.extend(pre_labels)
                    all_dev_labels.extend(label)

            print(Counter(all_dev_predictions))
            print(Counter(all_dev_labels))
            print('-------------------------------------------')
            print('dev集的结果：')
            score_f1 = evaluate(all_dev_predictions, all_dev_labels)
            print('-------------------------------------------')
            if score_f1 >= best_f1:
                print('保存模型')
                best_f1 = score_f1
                torch.save(model.state_dict(), PATH)


I0415 13:49:16.252601 140541114541824 configuration_utils.py:281] loading configuration file /home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/bert_config.json
I0415 13:49:16.254593 140541114541824 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "m

(99913, 6)


fold：0__epoch：0__train:   0%| | 0/21410 [00:00<?, ?it/s]

327623683
Counter({0: 2, 1: 1, 2: 1})
Counter({1: 3, 2: 1})
	precison_score： [0.         0.33333333 0.        ]
	recall_score： [0. 1. 0.]
	f1_score： [0.  0.5 0. ]
	f1_score： 0.16666666666666666


/usr/local/miniconda3/envs/dl10/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/miniconda3/envs/dl10/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
fold：0__epoch：0__train:   0%| | 51/21410 [00:36<4:11:54,  1.41it/s]

...

fold：2__epoch：2__train:   3%| | 551/21410 [06:42<4:18:41,  1.34it/s]

Counter({1: 1381, 2: 489, 0: 334})
Counter({1: 1273, 2: 534, 0: 397})
	precison_score： [0.64987406 0.87195601 0.72659176]
	recall_score： [0.77245509 0.80376539 0.79345603]
	f1_score： [0.70588235 0.83647325 0.75855327]
	f1_score： 0.7669696251837133


fold：2__epoch：2__train:   3%| | 601/21410 [07:19<4:19:43,  1.34it/s]

Counter({1: 1505, 2: 541, 0: 358})
Counter({1: 1386, 2: 591, 0: 427})
	precison_score： [0.64871194 0.87301587 0.73096447]
	recall_score： [0.77374302 0.80398671 0.79852126]
	f1_score： [0.70573248 0.83708059 0.76325088]
	f1_score： 0.7686879874728345


fold：2__epoch：2__train:   3%| | 613/21410 [07:27<4:13:39,  1.37it/s]

In [ ]:
for name,_ in model.named_parameters():
    print(name)

# predict_test

In [9]:
df_test = pd.read_csv(r'./corpus/df_test.csv')
# 测试集df_test
test_dataset = CLSDataset(df_test, is_train=False)
test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              batch_size=24,
                              collate_fn=collate_dev_test_fn)

## 5-flod

In [17]:
model = CLS_model(embedding_dim = 1024, target_size = 3).to(device)
result = pd.DataFrame()
for model_pkl in os.listdir('output'):
    if 'bert_large_pag' in model_pkl: 
        PATH = os.path.join('output', model_pkl)
        model.load_state_dict(torch.load(PATH))
        model.eval()
        
        all_ids = []
        all_predicionts = []
        with torch.no_grad():
            pbar = tqdm.tqdm(enumerate(test_dataloader), 
                     total=len(test_dataloader), 
                     ncols=50)
            with  pbar as t:
                for idx, batch_samples in t:
                    query_id = batch_samples['query_id']             
                    input_ids = batch_samples['input_ids']
                    token_type_ids = batch_samples['token_type_ids']
                    attention_mask = batch_samples['attention_mask']

                    #
                    input_ids = torch.LongTensor(input_ids).to(device)
                    token_type_ids = torch.LongTensor(token_type_ids).to(device)
                    attention_mask = torch.LongTensor(attention_mask).to(device)

                    # 前向传播
                    predictions = model(input_ids, 
                                       token_type_ids = token_type_ids,
                                       attention_mask = attention_mask)

                    predictions = torch.max(predictions, dim=1)[1]
                    predictions = predictions.cpu().numpy().tolist()

                    all_ids.extend(query_id)
                    all_predicionts.extend(predictions)
                    
                result['ID'] = all_ids
                result[PATH] = all_predicionts

I0417 02:45:12.003535 140541114541824 configuration_utils.py:281] loading configuration file /home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/bert_config.json
I0417 02:45:12.006395 140541114541824 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "m

In [30]:
t = result.copy()

In [31]:
t['prediction'] = t.mode(axis=1,numeric_only=True)[0].astype(int)
t.rename({'ID':'测试数据id', "prediction":"情感极性"}, inplace=True, axis=1)

In [32]:
t.head()

,测试数据id,output/bert_large_pag_1.pkl,output/bert_large_pag_0.pkl,output/bert_large_pag_2.pkl,情感极性
0,4456068992182160,1,1,1,1
1,4456424178427250,0,0,0,0
2,4456797466940200,1,1,0,1
3,4456791021108920,0,0,0,0
4,4457086404997440,0,0,0,0


In [33]:
t['情感极性'] = t['情感极性'].map({0:-1, 1:0, 2:1})
t.情感极性.value_counts()

 0    6036
 1    2461
-1    1503
Name: 情感极性, dtype: int64

In [36]:
t[['测试数据id', '情感极性']].to_csv('result.csv', index=False)

## single

In [37]:
model = CLS_model(embedding_dim = 1024, target_size = 3).to(device)
result = pd.DataFrame()

model.load_state_dict(torch.load('./output/bert_large_pag_1.pkl'))
model = model.cuda()
model.eval()

all_ids = []
all_predicionts = []
with torch.no_grad():
    pbar = tqdm.tqdm(enumerate(test_dataloader), 
                     total=len(test_dataloader), 
                     ncols=50)
    with  pbar as t:
        for idx, batch_samples in t:
            query_id = batch_samples['query_id']             
            input_ids = batch_samples['input_ids']
            token_type_ids = batch_samples['token_type_ids']
            attention_mask = batch_samples['attention_mask']

            #
            input_ids = torch.LongTensor(input_ids).to(device)
            token_type_ids = torch.LongTensor(token_type_ids).to(device)
            attention_mask = torch.LongTensor(attention_mask).to(device)

            # 前向传播
            predictions = model(input_ids, 
                               token_type_ids = token_type_ids,
                               attention_mask = attention_mask)
            
            predictions = torch.max(predictions, dim=1)[1]
            predictions = predictions.cpu().numpy().tolist()

            all_ids.extend(query_id)
            all_predicionts.extend(predictions)
        result['测试数据id'] = all_ids
        result['情感极性'] = all_predicionts

I0417 03:05:01.270222 140541114541824 configuration_utils.py:281] loading configuration file /home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/bert_config.json
I0417 03:05:01.272754 140541114541824 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "m

In [38]:
result['情感极性'] = result['情感极性'].map({0:-1, 1:0, 2:1})
result.情感极性.value_counts()

 0    6068
 1    2504
-1    1428
Name: 情感极性, dtype: int64

In [39]:
result.to_csv('./result.csv', index=False)

In [41]:
result.head()

,测试数据id,情感极性
0,4456068992182160,0
1,4456424178427250,-1
2,4456797466940200,0
3,4456791021108920,-1
4,4457086404997440,-1


## predict

In [46]:
model = CLS_model(embedding_dim = 1024, target_size = 3).to(device)
model.load_state_dict(torch.load('./output/bert_large_pag_1.pkl'))
model = model.cuda()
model.eval()


I0417 03:46:48.754824 140541114541824 configuration_utils.py:281] loading configuration file /home/zhoujx/Pretrained_models/chinese_roberta_wwm_large_ext_pytorch/bert_config.json
I0417 03:46:48.756883 140541114541824 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "m

CLS_model(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_a

In [52]:
model.predict('今天去运动了')

'中立'